# Cross Language Information Retrieval

## Housekeeping: File encodings and tokenisation

In [76]:
from nltk.tokenize import word_tokenize
from __future__ import division #To properly handle floating point divisions.
import math

#Function to tokenise string/sentences.
def tokenize(line, tokenizer=word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

In [77]:
tokenize("Seit damals ist er auf über 10.000 Punkte gestiegen.")

['seit',
 'damals',
 'ist',
 'er',
 'auf',
 '\\xfcber',
 '10.000',
 'punkte',
 'gestiegen',
 '.']

In [78]:
DEVELOPMENT_DOCS = 'data/clir/devel.docs' #Data file for IR engine development

DEVELOPMENT_QUERIES = 'data/clir/devel.queries' #Data file containing queries in German

DEVELOPMENT_QREL = 'data/clir/devel.qrel' #Data file containing a relevance score or query-doc pairs

BITEXT_ENG = 'data/clir/bitext.en' #Bitext data file in English for translation engine and language model development

BITEXT_DE = 'data/clir/bitext.de' #Bitext data file in German

NEWSTEST_ENG = 'data/clir/newstest.en' #File for testing language model

## Information Retrieval using [Okapi BM25](https://en.wikipedia.org/wiki/Okapi_BM25)

In [79]:
import nltk
import re

stopwords = set(nltk.corpus.stopwords.words('english')) #converting stopwords to a set for faster processing in the future.
stemmer = nltk.stem.PorterStemmer() 

#Function to extract and tokenize terms from a document
def extract_and_tokenize_terms(doc):
    terms = []
    for token in tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are faster over sets than lists
            if not re.search(r'\d',token) and not re.search(r'[^A-Za-z-]',token): #Removing numbers and punctuations 
                #(excluding hyphenated words)
                terms.append(stemmer.stem(token.lower()))
    return terms

documents = {} #Dictionary to store documents with ids as keys.

In [80]:
#Reading each line in the file and storing it documents dictionary
f = open(DEVELOPMENT_DOCS)

for line in f:
    doc = line.split("\t")
    terms = extract_and_tokenize_terms(doc[1])
    documents[doc[0]] = terms
f.close()

In [81]:
documents['290'][:20] #To keep things short, we're only going to check out 20 tokens.

[u'name',
 u'plural',
 u'ae',
 u'first',
 u'letter',
 u'vowel',
 u'iso',
 u'basic',
 u'latin',
 u'alphabet',
 u'similar',
 u'ancient',
 u'greek',
 u'letter',
 u'alpha',
 u'deriv',
 u'upper',
 u'case',
 u'version',
 u'consist']

In [115]:
#Building an inverted index for the documents

from collections import defaultdict
    
inverted_index = defaultdict(set)

for docid, terms in documents.items():
    for term in terms:
        inverted_index[term].add(docid)    

In [116]:
inverted_index['pizza']

{'121569',
 '16553',
 '212541',
 '228211',
 '261023',
 '265975',
 '276433',
 '64083',
 '69930',
 '72701',
 '73441',
 '74323'}

In [117]:
#Building a TF-IDF representation using BM25 

NO_DOCS = len(documents) #Number of documents

AVG_LEN_DOC = sum([len(doc) for doc in documents.values()])/len(documents) #Average length of documents

#The function below takes the documentid, and the term, to calculate scores for the tf and idf
#components, and multiplies them together.
def tf_idf_score(k1,b,term,docid):  
    
    ft = len(inverted_index[term]) 
    term = stemmer.stem(term.lower())
    fdt =  documents[docid].count(term)
    
    idf_comp = math.log((NO_DOCS - ft + 0.5)/(ft+0.5))
    
    tf_comp = ((k1 + 1)*fdt)/(k1*((1-b) + b*(len(documents[docid])/AVG_LEN_DOC))+fdt)
    
    return idf_comp * tf_comp

#Function to create tf_idf matrix without the query component
def create_tf_idf(k1,b):
    tf_idf = defaultdict(dict)
    for term in set(inverted_index.keys()):
        for docid in inverted_index[term]:
            tf_idf[term][docid] = tf_idf_score(k1,b,term,docid)
    return tf_idf

In [118]:
#Creating tf_idf matrix with said parameter values: k1 and b for all documents.
tf_idf = create_tf_idf(1.5,0.5)

In [119]:
#Function to retrieve query component
def get_qtf_comp(k3,term,fqt):
    return ((k3+1)*fqt[term])/(k3 + fqt[term])


#Function to retrieve documents || Returns a set of documents and their relevance scores. 
def retr_docs(query,result_count):
    q_terms = [stemmer.stem(term.lower()) for term in query.split() if term not in stopwords] #Removing stopwords from queries
    fqt = {}
    for term in q_terms:
        fqt[term] = fqt.get(term,0) + 1
    
    scores = {}
    
    for word in fqt.keys():
        #print word + ': '+ str(inverted_index[word])
        for document in inverted_index[word]:
            scores[document] = scores.get(document,0) + (tf_idf[word][document]*get_qtf_comp(0,word,fqt)) #k3 chosen as 0 (default)
    
    return sorted(scores.items(),key = lambda x : x[1] , reverse=True)[:result_count]        

In [120]:
retr_docs("Manchester United",5)

[('19961', 12.570721363284687),
 ('83266', 12.500367334396838),
 ('266959', 12.46418348068098),
 ('20206', 12.324327863972716),
 ('253314', 12.008548114449386)]

In [121]:
documents['19961'][:30]

[u'manchest',
 u'unit',
 u'manchest',
 u'unit',
 u'footbal',
 u'club',
 u'english',
 u'profession',
 u'footbal',
 u'club',
 u'base',
 u'old',
 u'trafford',
 u'greater',
 u'manchest',
 u'play',
 u'premier',
 u'leagu',
 u'found',
 u'newton',
 u'heath',
 u'lyr',
 u'footbal',
 u'club',
 u'club',
 u'chang',
 u'name',
 u'manchest',
 u'unit',
 u'move']

### Language Model:

In [89]:
#Calculating the unigram, bigram and trigram counts. 

f = open(BITEXT_ENG)

train_sentences = []

for line in f:
    train_sentences.append(tokenize(line))

f.close()    

#Function to mark the first occurence of words as unknown, for training.
def check_for_unk_train(word,unigram_counts):
    if word in unigram_counts:
        return word
    else:
        unigram_counts[word] = 0
        return "UNK"

#Function to convert sentences for training the language model.    
def convert_sentence_train(sentence,unigram_counts):
    #<s1> and <s2> are sentinel tokens added to the start and end, for handling tri/bigrams at the start of a sentence.
    return ["<s1>"] + ["<s2>"] + [check_for_unk_train(token.lower(),unigram_counts) for token in sentence] + ["</s2>"]+ ["</s1>"]

#Function to obtain unigram, bigram and trigram counts.
def get_counts(sentences):
    trigram_counts = defaultdict(lambda: defaultdict(dict))
    bigram_counts = defaultdict(dict)
    unigram_counts = {}
    for sentence in sentences:
        sentence = convert_sentence_train(sentence, unigram_counts)
        for i in range(len(sentence) - 2):
            trigram_counts[sentence[i]][sentence[i+1]][sentence[i+2]] = trigram_counts[sentence[i]][sentence[i+1]].get(sentence[i+2],0) + 1
            bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
            unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1
    unigram_counts["</s1>"] = unigram_counts["<s1>"]
    unigram_counts["</s2>"] = unigram_counts["<s2>"]
    bigram_counts["</s2>"]["</s1>"] = bigram_counts["<s1>"]["<s2>"]
    return unigram_counts, bigram_counts, trigram_counts

In [90]:
unigram_counts, bigram_counts,trigram_counts = get_counts(train_sentences)

In [91]:
#Constructing unigram model with 'add-k' smoothing
token_count = sum(unigram_counts.values())

#Function to convert unknown words for testing. 
#Words that don't appear in the training corpus (even if they are in the test corpus) are marked as UNK.
def check_for_unk_test(word,unigram_counts):
    if word in unigram_counts and unigram_counts[word] > 0:
        return word
    else:
        return "UNK"


def convert_sentence_test(sentence,unigram_counts):
    return ["<s1>"] + ["<s2>"] + [check_for_unk_test(word.lower(),unigram_counts) for word in sentence] + ["</s2>"]  + ["</s1>"]

#Returns the log probability of a unigram, with add-k smoothing. We're taking logs to avoid probability underflow.
def get_log_prob_addk(word,unigram_counts,k):
    return math.log((unigram_counts[word] + k)/ \
                    (token_count + k*len(unigram_counts)))

#Returns the log probability of a sentence.
def get_sent_log_prob_addk(sentence, unigram_counts,k):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_addk(word, unigram_counts,k) for word in sentence])


def calculate_perplexity_uni(sentences,unigram_counts, token_count, k):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 2 # have to consider the end token
        total_log_prob += get_sent_log_prob_addk(sentence,unigram_counts,k)
    return math.exp(-total_log_prob/test_token_count)


f = open(NEWSTEST_ENG)

test_sents = []
for line in f:
    test_sents.append(tokenize(line))
f.close()

In [92]:
#Calculating the perplexity for different ks
ks = [0.0001,0.01,0.1,1,10]

for k in ks:
    print str(k) +": " + str(calculate_perplexity_uni(test_sents,unigram_counts,token_count,k))


0.0001: 613.918691403
0.01: 614.027477551
0.1: 615.06903252
1: 628.823994251
10: 823.302441447


In [93]:
#Calculating the N1/N paramaters for Trigrams/Bigrams/Unigrams in Katz-Backoff Smoothing

TRI_ONES = 0 #N1 for Trigrams
TRI_TOTAL = 0 #N for Trigrams

for twod in trigram_counts.values():
    for oned in twod.values():
        for val in oned.values():
            if val==1:
                TRI_ONES+=1 #Count of trigram seen once
            TRI_TOTAL += 1 #Count of all trigrams seen

BI_ONES = 0 #N1 for Bigrams
BI_TOTAL = 0 #N for Bigrams

for oned in bigram_counts.values():
    for val in oned.values():
        if val==1:
            BI_ONES += 1 #Count of bigram seen once
        BI_TOTAL += 1 #Count of all bigrams seen
        
UNI_ONES = unigram_counts.values().count(1)
UNI_TOTAL = len(unigram_counts)

In [94]:
#Constructing trigram model with backoff smoothing

TRI_ALPHA = TRI_ONES/TRI_TOTAL #Alpha parameter for trigram counts
    
BI_ALPHA = BI_ONES/BI_TOTAL #Alpha parameter for bigram counts

UNI_ALPHA = UNI_ONES/UNI_TOTAL
    
def get_log_prob_back(sentence,i,unigram_counts,bigram_counts,trigram_counts,token_count):
    if trigram_counts[sentence[i-2]][sentence[i-1]].get(sentence[i],0) > 0:
        return math.log((1-TRI_ALPHA)*trigram_counts[sentence[i-2]][sentence[i-1]].get(sentence[i])/bigram_counts[sentence[i-2]][sentence[i-1]])
    else:
        if bigram_counts[sentence[i-1]].get(sentence[i],0)>0:
            return math.log(TRI_ALPHA*((1-BI_ALPHA)*bigram_counts[sentence[i-1]][sentence[i]]/unigram_counts[sentence[i-1]]))
        else:
            return math.log(TRI_ALPHA*BI_ALPHA*(1-UNI_ALPHA)*((unigram_counts[sentence[i]]+0.0001)/(token_count+(0.0001)*len(unigram_counts)))) 
        
        
def get_sent_log_prob_back(sentence, unigram_counts, bigram_counts,trigram_counts, token_count):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_back(sentence,i, unigram_counts,bigram_counts,trigram_counts,token_count) for i in range(2,len(sentence))])


def calculate_perplexity_tri(sentences,unigram_counts,bigram_counts,trigram_counts, token_count):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 2 # have to consider the end token
        total_log_prob += get_sent_log_prob_back(sentence,unigram_counts,bigram_counts,trigram_counts,token_count)
    return math.exp(-total_log_prob/test_token_count)

In [95]:
#Calculating the perplexity 
calculate_perplexity_tri(test_sents,unigram_counts,bigram_counts,trigram_counts,token_count)

461.64686176451505

### Translation model

In [96]:
#Creating lists of English and German sentences from bitext.

from nltk.translate import IBMModel1
from nltk.translate import AlignedSent, Alignment

eng_sents = []
de_sents = []

f = open(BITEXT_ENG)
for line in f:
    terms = tokenize(line)
    eng_sents.append(terms)
f.close()

f = open(BITEXT_DE)
for line in f:
    terms = tokenize(line)
    de_sents.append(terms)
f.close()

In [97]:
#Zipping together the bitexts for easier access
paral_sents = zip(eng_sents,de_sents)

In [98]:
#Building English to German translation table for words (Backward alignment)
eng_de_bt = [AlignedSent(E,G) for E,G in paral_sents]
eng_de_m = IBMModel1(eng_de_bt, 5)

In [100]:
#Building German to English translation table for words (Backward alignment)
de_eng_bt = [AlignedSent(G,E) for E,G in paral_sents]
de_eng_m = IBMModel1(de_eng_bt, 5)

In [101]:
#Script below to combine alignments using set intersections
combined_align = []

for i in range(len(eng_de_bt)):

    forward = {x for x in eng_de_bt[i].alignment}
    back_reversed = {x[::-1] for x in de_eng_bt[i].alignment}
    
    combined_align.append(forward.intersection(back_reversed))

In [102]:
#Creating German to English dictionary with occurence count of word pairs
de_eng_count = defaultdict(dict)

for i in range(len(de_eng_bt)):
    for item in combined_align[i]:
        de_eng_count[de_eng_bt[i].words[item[1]]][de_eng_bt[i].mots[item[0]]] =  de_eng_count[de_eng_bt[i].words[item[1]]].get(de_eng_bt[i].mots[item[0]],0) + 1

In [103]:
#Creating a English to German dict with occ count of word pais
eng_de_count = defaultdict(dict)

for i in range(len(eng_de_bt)):
    for item in combined_align[i]:
        eng_de_count[eng_de_bt[i].words[item[0]]][eng_de_bt[i].mots[item[1]]] =  eng_de_count[eng_de_bt[i].words[item[0]]].get(eng_de_bt[i].mots[item[1]],0) + 1

In [104]:
#Creating German to English table with word translation probabilities
de_eng_prob = defaultdict(dict)

for de in de_eng_count.keys():
    for eng in de_eng_count[de].keys():
        de_eng_prob[de][eng] = de_eng_count[de][eng]/sum(de_eng_count[de].values())

In [105]:
#Creating English to German dict with word translation probabilities 
eng_de_prob = defaultdict(dict)

for eng in eng_de_count.keys():
    for de in eng_de_count[eng].keys():
        eng_de_prob[eng][de] = eng_de_count[eng][de]/sum(eng_de_count[eng].values())

In [106]:
#Examples of translating individual words from German to English
print de_eng_prob['frage']

print de_eng_prob['handlung']

print de_eng_prob['haus']


{'question': 0.970873786407767, 'issue': 0.019417475728155338, 'matter': 0.009708737864077669}
{'rush': 1.0}
{'begins': 0.058823529411764705, 'house': 0.9411764705882353}


In [107]:
#Building noisy channel translation model
def de_eng_noisy(german):
    noisy={}
    for eng in de_eng_prob[german].keys():
        noisy[eng] = eng_de_prob[eng][german]+ get_log_prob_addk(eng,unigram_counts,0.0001)
    return noisy

In [108]:
#Test block to check alignments
print de_eng_noisy('vater')
print de_eng_noisy('haus')
print de_eng_noisy('das')
print de_eng_noisy('entschuldigung')

{'father': -8.798834996562721}
{'begins': -10.2208672198799, 'house': -8.163007778647888}
{'this': -5.214590799418497, 'the': -3.071527829335362, 'that': -4.664995720177421}
{'excuse': -11.870404868087332, 'apology': -12.39683538573032, 'comprehend': -11.89683538573032}


In [109]:
eng_de_prob['sorry']

{'bereue': 1.0}

In [122]:
#Translating first 5 queries into English

#Function for direct translation
def de_eng_direct(query):
    query_english = [] 
    query_tokens = tokenize(query)
    
    for token in query_tokens:
        try:
            query_english.append(max(de_eng_prob[token], key=de_eng_prob[token].get))
        except:
            query_english.append(token) #Returning the token itself when it cannot be found in the translation table.
            #query_english.append("NA") 
    
    return " ".join(query_english)

#Function for noisy channel translation
def de_eng_noisy_translate(query):  
    query_english = [] 
    query_tokens = tokenize(query)
    
    for token in query_tokens:
        try:
            query_english.append(max(de_eng_noisy(token), key=de_eng_noisy(token).get))
        except:
            query_english.append(token) #Returning the token itself when it cannot be found in the translation table.
            #query_english.append("NA") 
    
    return " ".join(query_english)
            
f = open(DEVELOPMENT_QUERIES)

lno = 0
plno = 0

#Also building a dictionary of query ids and query content (only for the first 100s)
german_qs = {}

test_query_trans_sents = [] #Building a list for perplexity checks.

for line in f:
    lno+=1
    query_id = line.split('\t')[0]
    query_german = line.split('\t')[1]  
    
    german_qs[query_id] = query_german.strip()
    
    translation = str(de_eng_noisy_translate(query_german))
 
    if plno<5:
        print query_id + "\n" + "German: " + str(query_german) + "\n" + "English: " + translation +"\n\n"
        plno+=1
    test_query_trans_sents.append(translation)
    if lno==100:
        break

f.close()

82
German: der ( von engl . action : tat , handlung , bewegung ) ist ein filmgenre des unterhaltungskinos , in welchem der fortgang der äußeren handlung von zumeist spektakulär inszenierten kampf - und gewaltszenen vorangetrieben und illustriert wird .

English: the ( , guises . action : indeed , rush , movement ) is a filmgenre the unterhaltungskinos , in much the fortgang the external rush , zumeist spektakul\xe4r inszenierten fight - and gewaltszenen pushed and illustriert will .


116
German: die ( einheitenzeichen : u für unified atomic mass unit , veraltet amu für atomic mass unit ) ist eine maßeinheit der masse .

English: the ( einheitenzeichen : u for unified atomic mass unit , obsolete amu for atomic mass unit ) is a befuddled the mass .


240
German: der von lateinisch actualis , " wirklich " , auch aktualitätsprinzip , uniformitäts - oder gleichförmigkeitsprinzip , englisch uniformitarianism , ist die grundlegende wissenschaftliche methode in der .

English: the , lateinisc

### Combining, and Evaluation

In [123]:
#Building a dictionary for queryids and relevant document ids
qrel = defaultdict(list)

f = open(DEVELOPMENT_QREL)

for line in f:
    item = line.split('\t')
    qrel[item[0]].append(item[2])
    
f.close()

In [124]:
#Single function to retreive documents for a German query
def trans_retr_docs(german_query,no_of_results,translation_function):
    
    trans_query = " ".join(extract_and_tokenize_terms(translation_function(german_query)))
    return [item[0] for item in retr_docs(trans_query,no_of_results)] #Retriving 100 documents

#Calculating the map score
def calc_map(no_of_results,translation_function):
    
    average_precision = []
    
    for gq in german_qs.keys():
        
        relevant_docs = qrel[gq]
        incremental_precision = []
        resulting_docs = trans_retr_docs(german_qs[gq],no_of_results,translation_function)
        
        total_counter = 0
        true_positive_counter = 0
        
        for doc in resulting_docs:
            total_counter+=1
            if doc in relevant_docs:
                true_positive_counter += 1
                incremental_precision.append(true_positive_counter/total_counter)
        
        #For no relevant retreivals, the average precision will be considered 0.
        try:
            average_precision.append(sum(incremental_precision)/len(incremental_precision))
        except:
            average_precision.append(0)
        
    return (sum(average_precision)/len(average_precision))

In [125]:
#Printing the map score for direct translations
print calc_map(100,de_eng_direct)

0.356571675599


In [126]:
#Printing the map score for noisy channel translations
print calc_map(100,de_eng_noisy_translate)

0.364795198505
